In [1]:
import pandas as pd
import numpy as np
import gzip


# Import sets

In [2]:
path = "C:/Users/sobot/Downloads/DNA-leading-strand-main/DNA-leading-strand-main/"
file = "fullBacteriaP1.txt"

fullBacteriaP1line = pd.read_csv(path+file, sep=',', header=None)

file = "fullBacteriaP2.txt"
fullBacteriaP2line = pd.read_csv(path+file, sep=',', header=None)


# organize sets to become matrices with entries type float64

In [3]:
fullBacteriaP1line = fullBacteriaP1line.rename(columns={0:"column"})
fullBacteriaP2line = fullBacteriaP2line.rename(columns={0:"column"})

df1=fullBacteriaP1line.column.str.split(pat=' ',expand=True)
df2=fullBacteriaP2line.column.str.split(pat=' ',expand=True)

df1 = df1.astype(float)
df2 = df2.astype(float)

line=df1.iloc[:1]
matrix1=np.array(line).reshape(4,4)

fullBacteriaP1=[]
fullBacteriaP2=[]


for i in list(range(0,9222)):
    line=df1.iloc[i:i+1]
    matrix=np.array(line).reshape(4,4)
    fullBacteriaP1.append(matrix)
    
    line=df2.iloc[i:i+1]
    matrix=np.array(line).reshape(4,4)
    fullBacteriaP2.append(matrix)

fullBacteriaP1line

,column
0,0.3211370672670980 0.1298366236446730 0.284628...
1,0.3191806615776080 0.1953689567430030 0.267221...
2,0.3848945828642780 0.1566695557171650 0.163762...
3,0.4206054630864700 0.1566239440159290 0.179222...
4,0.3476212679068120 0.1868820039551750 0.204680...
...,...
9217,0.2082055436606680 0.2951218100804350 0.252994...
9218,0.1873094834170310 0.3050901299737080 0.279277...
9219,0.1824403786462050 0.3087933062137310 0.269243...
9220,0.1784117779043480 0.3186843059079380 0.257102...


# Find pi

In [4]:
pi1=[]
pi2=[]

for M in fullBacteriaP1:
    transposta = np.transpose(M)
    b,r = np.linalg.eig(transposta)
    vetor=r[:,0]/(r[0,0]+r[1,0]+r[2,0]+r[3,0])
    pi1.append(vetor)

for M in fullBacteriaP2:
    transposta = np.transpose(M)
    b,r = np.linalg.eig(transposta)
    vetor=r[:,0]/(r[0,0]+r[1,0]+r[2,0]+r[3,0])
    pi2.append(vetor)
    


# is the least DNA a leading strand?

In [90]:
# In order to know if is a leading strand, we need to verify if piG + piT > piA + piC, 
# where pi = [piA piC piG piT] was the eingenvectors found previously
# it returns a 9222x2:
# data frame 'classificao', where both fullBacteriaP1 and fullBacteriaP2 is classified as leading strand if GT>AC
# data frame 'classificao1only', listing all the sequences of fullBacteriaP1 with GT>AC (classified as leading strand) for which their corrrespondent ones in fullBacteriaP2 present GT<AC 
# data frame 'classificao2only', listing all the sequences of fullBacteriaP2 with GT>AC (classified as leading strand) for which their corrrespondent ones in fullBacteriaP1 present GT<AC 
# data frame 'naoclassificado', where both fullBacteriaP1 and fullBacteriaP2 were not classified as leading strand (GT<AC)
# data frame 'duploclassificado', where both fullBacteriaP1 and fullBacteriaP2 were classified as leading strand (GT>AC)

In [39]:
#classificacao=np.zeros((9222,2), dtype=np.int8)

classificacao1=[]
classificacao2=[]

for i in list(range(0,9222)):
    GT = pi1[i][2]+pi1[i][3]
    AC = pi1[i][0]+pi1[i][1]
    if GT>AC:
        classificacao1.append("leading")
    else:
        classificacao1.append("")

for i in list(range(0,9222)):
    GT = pi2[i][2]+pi2[i][3]
    AC = pi2[i][0]+pi2[i][1]
    if GT>AC:
        classificacao2.append("leading")
    else:
        classificacao2.append("")

d = {'fullBacteriaP1' : pd.Series(classificacao1), 
   'fullBacteriaP2' : pd.Series(classificacao2)}
        
classificacao= pd.DataFrame(d)

classificacao1only=classificacao
classificacao2only=classificacao
naoclassificado=classificacao
duploclassificado=classificacao

list1=['leading']
classificacao1only = classificacao1only[classificacao1only.fullBacteriaP1.isin(list1) == True]
classificacao1only = classificacao1only[classificacao1only.fullBacteriaP2.isin(list1) == False]

classificacao2only = classificacao2only[classificacao2only.fullBacteriaP1.isin(list1) == False]
classificacao2only = classificacao2only[classificacao2only.fullBacteriaP2.isin(list1) == True]

naoclassificado = naoclassificado[naoclassificado.fullBacteriaP1.isin(list1) == False]
naoclassificado = naoclassificado[naoclassificado.fullBacteriaP2.isin(list1) == False]

duploclassificado = duploclassificado[duploclassificado.fullBacteriaP1.isin(list1) == True]
duploclassificado = duploclassificado[duploclassificado.fullBacteriaP2.isin(list1) == True]



In [40]:
classificacao

,fullBacteriaP1,fullBacteriaP2
0,leading,
1,leading,
2,leading,
3,,leading
4,,leading
...,...,...
9217,leading,leading
9218,leading,
9219,leading,
9220,leading,


In [42]:
classificacao1only

,fullBacteriaP1,fullBacteriaP2
0,leading,
1,leading,
2,leading,
6,leading,
8,leading,
...,...,...
9216,leading,
9218,leading,
9219,leading,
9220,leading,


In [43]:
classificacao2only

,fullBacteriaP1,fullBacteriaP2
3,,leading
4,,leading
5,,leading
9,,leading
20,,leading
...,...,...
9182,,leading
9184,,leading
9185,,leading
9188,,leading


In [44]:
naoclassificado

,fullBacteriaP1,fullBacteriaP2
7,,
12,,
13,,
14,,
15,,
...,...,...
9145,,
9150,,
9181,,
9183,,


In [45]:
duploclassificado

,fullBacteriaP1,fullBacteriaP2
36,leading,leading
37,leading,leading
43,leading,leading
44,leading,leading
49,leading,leading
...,...,...
9177,leading,leading
9187,leading,leading
9194,leading,leading
9199,leading,leading
